<a href="https://colab.research.google.com/github/bigcarrot0722/kobert_emotion_analysis/blob/main/Two_Feeling(hate_ang)_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Kobert Package Download



In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

In [ ]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

## Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/PBL')

## Tokenizer, BertModel, Vocabulary 불러오기

In [ ]:
from kobert_tokenizer import KoBERTTokenizer 
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

In [ ]:
import torch
from transformers import BertModel

In [ ]:
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path, return_dict=False)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [ ]:
import gluonnlp as nlp
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

## Dataset 불러오기 & Train, Valid, Test로 Dataset 분할(8:1:1)

In [ ]:
import pandas as pd
feeling = pd.read_excel('feeling2.xlsx')

In [ ]:
feeling.loc[(feeling['Emotion'] == '공포'), 'Emotion'] = 0
feeling.loc[(feeling['Emotion'] == '슬픔'), 'Emotion'] = 1
feeling.loc[(feeling['Emotion'] == '중립'), 'Emotion'] = 2
feeling.loc[(feeling['Emotion'] == '놀람'), 'Emotion'] = 3
feeling.loc[(feeling['Emotion'] == '행복'), 'Emotion'] = 4
feeling.loc[(feeling['Emotion'] == '혐오'), 'Emotion'] = 5
feeling.loc[(feeling['Emotion'] == '분노'), 'Emotion'] = 6

In [ ]:
data_list = []
for a, label in zip(feeling['Sentence'], feeling['Emotion']):
  data = []
  data.append(a)
  data.append(str(label))

  data_list.append(data)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_list, test_size = 0.2, random_state = 0)
test_data, valid_data = train_test_split(test_data, test_size = 0.5, random_state = 0)

In [ ]:
train_hate = []
train_ang = []
for a, label in train_data:
  data = []
  if label == '6' or label == '5':
    if label == '5':
      data.append(a)
      data.append(str(0))
      train_hate.append(data)
    else:
      data.append(a)
      data.append(str(1))
      train_ang.append(data)


In [ ]:
train_data = train_hate + train_ang

In [ ]:
test_hate = []
test_ang = []
for a, label in test_data:
  data = []
  if label == '6' or label == '5':
    if label == '5':
      data.append(a)
      data.append(str(0))
      test_hate.append(data)
    else:
      data.append(a)
      data.append(str(1))
      test_ang.append(data)

In [ ]:
test_data = test_hate + test_ang

In [ ]:
valid_hate = []
valid_ang = []
for a, label in valid_data:
  data = []
  if label == '6' or label == '5':
    if label == '5':
      data.append(a)
      data.append(str(0))
      valid_hate.append(data)
    else:
      data.append(a)
      data.append(str(1))
      valid_ang.append(data)

In [ ]:
valid_data = valid_hate + valid_ang

In [ ]:
import random
random.shuffle(train_data)
random.shuffle(test_data)
random.shuffle(valid_data)

## BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## Setting parameters

In [ ]:
max_len = 64
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tok = tokenizer.tokenize


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

## batah size로 데이터 묶어주기

In [ ]:
data_train = BERTDataset(train_data, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(test_data, 0, 1, tok, vocab, max_len, True, False)
data_valid = BERTDataset(valid_data, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
device = torch.device("cuda:0")

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## Model training

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    print("1")
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
model.eval()
test_acc = 0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
print(test_acc)
print("test acc {}".format(test_acc / (batch_id+1)))